### California housing data

Compiled from census data in 1990.  See [data description](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) for more information.



In [18]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

housing_data = pd.read_csv(
    "https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

# Reindex and scale median house value
housing_data = housing_data.reindex(
    np.random.permutation(housing_data.index))
housing_data["median_house_value"] /= 1000.0
#housing_data

#housing_data.describe()

#plt.plot(housing_data['median_income'], housing_data['median_house_value'], 'k.')

# Adding synthetic data
housing_data["rooms_per_person"] = housing_data["total_rooms"] / housing_data["population"]
housing_data.corr()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_person
longitude,1.0,-0.9,-0.1,0.0,0.1,0.1,0.1,-0.0,-0.0,-0.1
latitude,-0.9,1.0,0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1,0.1
housing_median_age,-0.1,0.0,1.0,-0.4,-0.3,-0.3,-0.3,-0.1,0.1,-0.1
total_rooms,0.0,-0.0,-0.4,1.0,0.9,0.9,0.9,0.2,0.1,0.1
total_bedrooms,0.1,-0.1,-0.3,0.9,1.0,0.9,1.0,-0.0,0.0,0.0
population,0.1,-0.1,-0.3,0.9,0.9,1.0,0.9,-0.0,-0.0,-0.1
households,0.1,-0.1,-0.3,0.9,1.0,0.9,1.0,0.0,0.1,-0.0
median_income,-0.0,-0.1,-0.1,0.2,-0.0,-0.0,0.0,1.0,0.7,0.2
median_house_value,-0.0,-0.1,0.1,0.1,0.0,-0.0,0.1,0.7,1.0,0.2
rooms_per_person,-0.1,0.1,-0.1,0.1,0.0,-0.1,-0.0,0.2,0.2,1.0
